<a href="https://colab.research.google.com/github/ajbrittle1975/fictional-memory/blob/main/Keras_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Check Architechture

In [13]:
# %reset

In [14]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Thu Feb  9 19:59:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    28W /  70W |    760MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Install Necessary Libraries

In [15]:
url = 'https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files'
ext = '0.4.0-oneiric1_amd64.deb -qO'
!wget $url/libta-lib0_$ext libta.deb
!wget $url/ta-lib0-dev_$ext ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib
import talib
!pip install numpy pandas matplotlib tensorflow tensorflow-datasets keras keras-tuner alpha-vantage

(Reading database ... 128074 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.9.1-1) ...
Processing triggers for libc-bin (2.31-0ubuntu9.9) ...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Import Libraries

In [16]:
import math

import numpy as np # linear algebra
from numpy import array
from numpy import hstack

import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas import DataFrame as df
from pandas import concat

from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

from keras import layers
import keras_tuner as kt
from keras.models import Sequential 
from keras.layers import Activation
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.utils import timeseries_dataset_from_array

from talib import abstract

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators

print("All libraries loaded")

All libraries loaded


# Stock Configurations

In [30]:
config = {
    "alpha_vantage": {
        # Claim your free API key here: https://www.alphavantage.co/support/#api-key
        "key": "T7GH3GWEIV1DKHB0",
        "symbol": "TQQQ",
        "outputsize": "full",
        "key_adjusted_close": "5. adjusted close",
        "key_high": "2. high",
        "key_low": "3. low",
        "key_volume": "6. volume",
    },
    "data": {
        "window_size": 25,
        "train_split_size": 0.80,
        "dynamic_calcs": False,
    },
    "plots": {
        "xticks_interval": 90,  # show a date every 90 days
        "color_actual": "#001f3f",
        "color_train": "#3D9970",
        "color_val": "#0074D9",
        "color_pred_train": "#3D9970",
        "color_pred_val": "#0074D9",
        "color_pred_test": "#FF4136",
    },
    "model": {
        "input_size": 11,  # we are using 11 features
        "num_lstm_layers": 2,
        "lstm_size": 120,
        "dropout": 0.20,
    },
    "training": {
        "device": "cuda",  # "cuda" or "cpu"
        "batch_size": 25,
        "num_epoch": 200,
        "learning_rate": 0.01,
        "scheduler_step_size": 40,
    },
}


# Define Necessary Functions

In [31]:
def remove_nan_columns(data_array, data_date):
    # return data_array[~np.isnan(data_array).any(axis=1), :]
    nan_rows = np.argwhere(np.isnan(data_array).any(axis=1))
    print("Removing {0} nan rows from data stack...".format(len(nan_rows)))
    data_array = np.delete(data_array, nan_rows, 0)

    # given index of elements
    # remove largest indices first to not change length
    nan_row_list = list(nan_rows.flatten())
    data_date = list(data_date)
    for ele in sorted(nan_row_list, reverse=True):
        data_date.remove(data_date[ele])

    return data_array, data_date


class Normalizer:
    def __init__(self):
        self.mu = None
        self.sd = None

    def fit_transform(self, x):
        self.mu = np.nanmean(x, axis=(0), keepdims=True)
        self.sd = np.nanstd(x, axis=(0), keepdims=True)
        normalized_x = (x - self.mu) / self.sd
        return normalized_x

    def inverse_transform(self, x):
        return (x * self.sd) + self.mu


def download_data(config):
    ts = TimeSeries(key=config["alpha_vantage"]["key"])
    ti = TechIndicators(key=config["alpha_vantage"]["key"])
    dynamic_calcs = config["data"]["dynamic_calcs"] ## if we want to use dynamic calculation of indicators this is true

    print("Obtaining data for symbol: {0}".format(config["alpha_vantage"]["symbol"]))
    data, data_meta_data = ts.get_daily_adjusted(
        config["alpha_vantage"]["symbol"],
        outputsize=config["alpha_vantage"]["outputsize"],
    )

    print("Sorting data...")

    data_date = [date for date in data.keys()]
    data_date.reverse()

    print("Sorting close price...")
    data_close_price = [
        float(data[date][config["alpha_vantage"]["key_adjusted_close"]])
        for date in data.keys()
    ]
    data_close_price.reverse()
    data_close_price = np.array(data_close_price)
    data_price_pct_change = np.diff(data_close_price) / data_close_price[:-1] * 100
    data_price_pct_change = np.pad(data_price_pct_change, (1, 0), "constant")
    data_price_pct_change = data_price_pct_change.reshape(-1, 1)
    ## Normalize price percent change
    print("Normalizing price percent change data...")
    price_pct_change_scaler = MinMaxScaler()
    data_price_pct_change_norm = price_pct_change_scaler.fit_transform(
        data_price_pct_change
    )

    print("Sorting high prices...")
    data_high_price = [
        float(data[date][config["alpha_vantage"]["key_high"]]) for date in data.keys()
    ]
    data_high_price.reverse()
    data_high_price = np.array(data_high_price)

    print("Sorting low prices...")
    data_low_price = [
        float(data[date][config["alpha_vantage"]["key_low"]]) for date in data.keys()
    ]
    data_low_price.reverse()
    data_low_price = np.array(data_low_price)

    print("Sorting volume...")
    data_volume = [
        float(data[date][config["alpha_vantage"]["key_volume"]]) for date in data.keys()
    ]
    data_volume.reverse()
    data_volume = np.array(data_volume)
    # Calculate OBV before reshaping data
    print("Calculating OBV on balance volume...")
    obv_data = abstract.OBV(data_close_price, data_volume)
    data_volume = data_volume.reshape(-1, 1)
    ## Normalize Volume
    print("Normalizing Volume Data...")
    volume_scaler = Normalizer()
    volume_norm = volume_scaler.fit_transform(data_volume)

    print("Calculating HT_TRENDLINE...")
    ## Get HT Trend data
    ht_trend = abstract.HT_TRENDLINE(data_close_price)
    ht_trend = ht_trend.reshape(-1, 1)

    ## Normalize HT Trend
    print("Normalizing HT_TRENDLINE data...")
    ht_trendline_scaler = MinMaxScaler()
    ht_trendline_norm = ht_trendline_scaler.fit_transform(ht_trend)
    
    print("Normalizing On Balance Volume data...")
    ## Normalize OBV
    obv_norm = obv_data / data_volume
    obv_data = obv_data.reshape(-1, 1)
    obv_scaler = Normalizer()
    obv_norm = obv_scaler.fit_transform(obv_data)  

    print("Calculating MESA average price...")
    ## Get MESA average price
    mama, fama = abstract.MAMA(data_close_price, fastlimit=0.5, slowlimit=0.05)
    mesa_hist = mama - fama
    mesa_hist = mesa_hist.reshape(-1, 1)
    ## Normalize MESA
    print("Normalizing dynamic MESA trend data...")
    mesa_hist_scaler = Normalizer()
    mesa_hist_norm = mesa_hist_scaler.fit_transform(mesa_hist)

    print("Calculating HT_TREND sine signals...")
    ## Get HT_SINE signals
    sine, leadsine = abstract.HT_SINE(data_close_price)
    trend = abstract.HT_TRENDMODE(data_close_price)
    ht_trend_sine = (sine - leadsine) * trend
    ht_trend_sine = ht_trend_sine.reshape(-1, 1)
    ## Normalize HT_TREND
    print("Normalizing dynamic MACD histogram data...")
    ht_trend_sine_scaler = Normalizer()
    ht_trend_sine_norm = ht_trend_sine_scaler.fit_transform(ht_trend_sine)

    print("Calculating HT_PERIODS for dynamic indicators...")
    # Get period of data close price
    ht_period = abstract.HT_DCPERIOD(data_close_price)
    ht_period = np.around(ht_period)  # the instant period is optimal for oscillators

    print("Provisioning arrays for dynamic indicators...")
    # Provision arrays for additional indicators
    num_data_points = len(data_date)
    adx_ht = np.zeros(num_data_points, dtype=float)
    aroon_osc_ht = np.zeros(num_data_points, dtype=float)
    rsi_ht = np.zeros(num_data_points, dtype=float)
    macd_ht = np.zeros(num_data_points, dtype=float)
    macdsignal_ht = np.zeros(num_data_points, dtype=float)
    macdhist_ht = np.zeros(num_data_points, dtype=float)
    slowk_ht = np.zeros(num_data_points, dtype=float)
    slowd_ht = np.zeros(num_data_points, dtype=float)
    bb_lowerband_ht = np.zeros(num_data_points, dtype=float)
    bb_upperband_ht = np.zeros(num_data_points, dtype=float)
    bb_pct_ht = np.zeros(num_data_points, dtype=float)
    ult_osc_ht = np.zeros(num_data_points, dtype=float)

    # Get frequency-adjusted ADX
    print("Calculating dynamic ADX...")
    if dynamic_calcs:
      i = 0
      for period in ht_period:
          if not math.isnan(period):
              period = int(period)
              slowperiod = (
                  period if period % 2 == 0 else period + 1
              )  # Ensure slow period is even
              adx = abstract.ADX(
                  data_high_price, data_low_price, data_close_price, timeperiod=slowperiod
              )
              adx_ht[i] = adx[i]

          i += 1
      else:
        adx_ht = abstract.ADX(
          data_high_price, data_low_price, data_close_price
          )

    adx_ht = adx_ht.reshape(-1, 1)
    ## Normalize ADX
    print("Normalizing dynamic ADX data...")
    adx_norm = adx_ht / 100

    # Get frequency-adjusted Aroon Oscillator
    print("Calculating dynamic AROONOSC...")
    if dynamic_calcs:
      i = 0
      for period in ht_period:
          if not math.isnan(period):
              period = int(period)
              slowperiod = (
                  period if period % 2 == 0 else period + 1
              )  # Ensure slow period is even
              aroon_osc = abstract.AROONOSC(
                  data_high_price, data_low_price, timeperiod=int(slowperiod/2)
              )
              aroon_osc_ht[i] = aroon_osc[i]

          i += 1
      else:
        aroon_osc_ht = abstract.AROONOSC(
            data_high_price, data_low_price
        )

    aroon_osc_ht = aroon_osc_ht.reshape(-1, 1)
    ## Normalize Aroon Oscillator
    print("Normalizing dynamic AROONOSC data...")
    aroon_osc_norm = (aroon_osc_ht + 100) / 200  # oscillates between +/- 100

    # Get frequency-adjusted MACD
    print("Calculating dynamic MACD...")
    if dynamic_calcs:
      i = 0
      for period in ht_period:
          if not math.isnan(period):
              period = int(period)
              slowperiod = (
                  period if period % 2 == 0 else period + 1
              )  # Ensure slow period is even
              fastperiod = int((slowperiod / 2) - 1)
              signalperiod = math.floor(0.75 * fastperiod)
              macd, macdsignal, macdhist = abstract.MACD(
                  data_close_price,
                  fastperiod=fastperiod,
                  slowperiod=slowperiod,
                  signalperiod=signalperiod,
              )
              macd_ht[i] = macd[i]
              macdsignal_ht[i] = macdsignal[i]
              macdhist_ht[i] = macdhist[i]

          i += 1

      else:
        macd_ht, macdsignal_ht, macdhist_ht = abstract.MACD(
            data_close_price
        )


    macdhist_ht = macdhist_ht.reshape(-1, 1)
    ## Normalize MACD
    print("Normalizing dynamic MACD histogram data...")
    macd_scaler = Normalizer()
    macd_norm = macd_scaler.fit_transform(macdhist_ht)

    # Get frequency-adjusted Bollinger Bands
    print("Calculating dynamic BBANDS...")
    if dynamic_calcs:
      i = 0
      for period in ht_period:
          if not math.isnan(period):
              period = int(period)
              slowperiod = (
                  period if period % 2 == 0 else period + 1
              )  # Ensure slow period is even
              bb_upperband, bb_middleband, bb_lowerband = abstract.BBANDS(
                  data_close_price,
                  timeperiod=slowperiod,
                  nbdevup=float(2),
                  nbdevdn=float(2),
                  matype=1,
              )
              bb_upperband_ht[i] = bb_upperband[i]
              bb_lowerband_ht[i] = bb_lowerband[i]
              bb_pct_ht[i] = (data_close_price[i] - bb_lowerband[i]) / (
                  bb_upperband[i] - bb_lowerband[i]
              )

          i += 1

      else:
        bb_upperband, bb_middleband, bb_lowerband = abstract.BBANDS(
            data_close_price
        )
        bb_pct_ht = (data_close_price - bb_lowerband) / (
            bb_upperband - bb_lowerband
        )

    bb_pct_ht = bb_pct_ht.reshape(-1, 1)
    ## Normalize MACD
    print("Normalizing dynamic BBANDS Percentage data...")
    bb_pct_scaler = Normalizer()
    bb_pct_norm = bb_pct_scaler.fit_transform(bb_pct_ht)

    # Get frequency-adjusted RSI
    print("Calculating dynamic RSI...")
    if dynamic_calcs:
      i = 0
      for period in ht_period:
          if not math.isnan(period):
              period = (
                  period / 2 if period % 2 == 0 else (period + 1) / 2
              )  # Ensure slow period is even and divide by 2
              period = int(period)
              rsi = abstract.RSI(
                  data_close_price,
                  timeperiod=period,
              )
              rsi_ht[i] = rsi[i]

          i += 1
      else:
          rsi_ht = abstract.RSI(
                  data_close_price,
              )
        

    rsi_ht = rsi_ht.reshape(-1, 1)
    ## Normalize RSI
    print("Normalizing dynamic RSI data...")
    rsi_norm = rsi_ht / 100

    # Get frequency-adjusted Stochastic
    print("Calculating dynamic Stochastic...")
    if dynamic_calcs:
      i = 0
      for period in ht_period:
          if not math.isnan(period):
              period = (
                  period / 2 if period % 2 == 0 else (period + 1) / 2
              )  # Ensure slow period is even and divide by 2
              period = int(period)
              slowk, slowd = abstract.STOCH(
                  data_high_price,
                  data_low_price,
                  data_close_price,
                  fastk_period=period,
                  slowk_period=3,
                  slowk_matype=1,
                  slowd_period=3,
                  slowd_matype=1,
              )
              slowk_ht[i] = slowk[i]
              slowd_ht[i] = slowd[i]

          i += 1
      else:
        slowk_ht, slowd_ht = abstract.STOCH(
          data_high_price,
          data_low_price,
          data_close_price,
        )

    stochastic_hist_ht = slowk_ht - slowd_ht
    stochastic_hist_ht = stochastic_hist_ht.reshape(-1, 1)
    ## Normalize dynamic stochastic histogram
    print("Normalizing dynamic stochastic histogram data...")
    stochastic_hist_scaler = Normalizer()
    stochastic_hist_norm = stochastic_hist_scaler.fit_transform(stochastic_hist_ht)
    
    # Get frequency-adjusted Ultimate Oscillator
    print("Calculating dynamic ULTOSC...")
    if dynamic_calcs:
      i = 0
      for period in ht_period:
          if not math.isnan(period):
              period = (
                  period / 2 if period % 2 == 0 else (period + 1) / 2
              )  # Ensure slow period is even and divide by 2
              period = int(period)
              ult_osc = abstract.ULTOSC(
                  data_high_price,
                  data_low_price,
                  data_close_price,
                  timeperiod1=period,
                  timeperiod2=period*2,
                  timeperiod3=period*4,
              )
              ult_osc_ht[i] = ult_osc[i]

          i += 1
      else:
        ult_osc_ht = abstract.ULTOSC(
            data_high_price,
            data_low_price,
            data_close_price,
        )

    ult_osc_ht = ult_osc_ht.reshape(-1, 1)
    ## Normalize dynamic stochastic histogram
    print("Normalizing dynamic ULTOSC data...")
    

    print("volume_norm: {0}".format(volume_norm.shape))
    print("obv_norm: {0}".format(obv_norm.shape))
    print("mesa_hist_norm: {0}".format(mesa_hist_norm.shape))
    print("ht_trend_sine: {0}".format(ht_trend_sine.shape))
    print("macd_norm: {0}".format(volume_norm.shape))
    print("adx_norm: {0}".format(adx_norm.shape))
    print("aroon_osc_norm: {0}".format(aroon_osc_norm.shape))
    print("bb_pct_norm: {0}".format(bb_pct_norm.shape))
    print("rsi_norm: {0}".format(rsi_norm.shape))
    print("ult_osc_ht: {0}".format(ult_osc_ht.shape))
    print("data_price_pct_change_norm: {0}".format(data_price_pct_change_norm.shape))



    data_stack = hstack(
        (
            volume_norm,
            obv_norm,
            mesa_hist_norm,
            ht_trend_sine,
            macd_norm,
            adx_norm,
            aroon_osc_norm,
            bb_pct_norm,
            rsi_norm,
            stochastic_hist_norm,
            ult_osc_ht,
            data_price_pct_change_norm,
        )
    )

    print(data_stack)
    # print(data_stack.size)
    data_stack, data_date = remove_nan_columns(data_stack, data_date)
    num_data_points = data_stack.shape[0]
    display_date_range = (
        "from " + data_date[0] + " to " + data_date[num_data_points - 1]
    )
    print("Number data points:", num_data_points, display_date_range)

    # print(data_stack)
    # print(data_stack.size)

    return data_date, data_stack, num_data_points, display_date_range, price_pct_change_scaler

# Download Stock Data

In [19]:
data_date, data_stack, num_data_points, display_date_range, price_pct_change_scaler = download_data(config)

# # Convert date data into a datetime object and pandas column
# df_date = pd.DataFrame(data_date, columns=['Date'])
# df_date = pd.to_datetime(df_date['Date'])

# # Create full pandas datastack indexed by date
# df = pd.DataFrame(
#             data_stack, 
#             columns = [  
#             "mesa_hist",          
#             "ht_trend_sine",
#             "macd_norm",
#             "adx_norm",
#             "aroon_osc_norm",
#             "bb_pct_norm",
#             "rsi_norm",
#             "stochastic_hist_norm",
#             "ult_osc_ht",
#             "data_price_pct_change_norm"], index = df_date
#             )


Obtaining data for symbol: TQQQ
Sorting data...
Sorting close price...
Normalizing price percent change data...
Sorting high prices...
Sorting low prices...
Sorting volume...
Calculating OBV on balance volume...
Normalizing Volume Data...
Calculating HT_TRENDLINE...
Normalizing HT_TRENDLINE data...
Normalizing On Balance Volume data...
Calculating MESA average price...
Normalizing dynamic MESA trend data...
Calculating HT_TREND sine signals...
Normalizing dynamic MACD histogram data...
Calculating HT_PERIODS for dynamic indicators...
Provisioning arrays for dynamic indicators...
Calculating dynamic ADX...
Normalizing dynamic ADX data...
Calculating dynamic AROONOSC...
Normalizing dynamic AROONOSC data...
Calculating dynamic MACD...
Normalizing dynamic MACD histogram data...
Calculating dynamic BBANDS...
Normalizing dynamic BBANDS Percentage data...
Calculating dynamic RSI...
Normalizing dynamic RSI data...
Calculating dynamic Stochastic...
Normalizing dynamic stochastic histogram data.

# Split Data Into Train and Test Groups

In [20]:
# Define parameters from config
batch_size = config["training"]["batch_size"]
epochs = config["training"]["num_epoch"]
dropout=config["model"]["dropout"]
window_size = config["data"]["window_size"]
features = config["model"]["input_size"]
neuron_per_feature = 8

# split into train and test sets
input_data = data_stack[:, :-1] # all but last column
target_data = data_stack[:, -1] # for last column
print(target_data)
target_data = np.pad(target_data,((1,0)), mode='constant')[:-1] # Shift so we are getting target for next day
target_data = np.reshape(target_data, (input_data.shape[0], 1) )
print(target_data)

dataset = keras.utils.timeseries_dataset_from_array(
    input_data, 
    target_data, 
    sequence_length=window_size, 
    batch_size=batch_size,
    seed=None)

batch_count = int(input_data.shape[0]/batch_size)

[0.51788744 0.39623752 0.57343519 ... 0.5193865  0.66159508 0.47333987]
[[0.        ]
 [0.51788744]
 [0.39623752]
 ...
 [0.47677962]
 [0.5193865 ]
 [0.66159508]]



### GRU Network

In [21]:
def build_model(hp):
  dnn_units_min, dnn_units_max = 32, 512
  active_func_ss = ['relu', 'tanh']
  optimizer_ss = ['adam']
  lr_min, lr_max = 1e-4, 1e-1
  
  active_func = hp.Choice('activation', active_func_ss)
  optimizer = hp.Choice('optimizer', optimizer_ss)
  lr = hp.Float('learning_rate', min_value=lr_min, max_value=lr_max, sampling='log')
  dropout_ss = hp.Choice("dropouts_", [0.1, 0.2, 0.3, 0.4, 0.5])
    
  # Build Model
  # Sequential
  model = tf.keras.Sequential()

  # First GRU Layer
  # input shape should be form (batch, window_size, features)
  npf1 = hp.Int(
      name='neuron_per_feature1',
      min_value=2,
      max_value=200,
      step=4,
      sampling=None,
      default=None,
      parent_name=None,
      parent_values=None,
  )
  model.add(tf.keras.layers.GRU(
      npf1, 
      input_shape=(window_size, features),
      activation = 'tanh',
      dropout=dropout_ss,
      return_sequences=True,
      name='gru1',)
      )
  
  # Second GRU Layer
  npf2 = hp.Int(
      name='neuron_per_feature2',
      min_value=2,
      max_value=200,
      step=2,
      sampling=None,
      default=None,
      parent_name=None,
      parent_values=None,
  )
  model.add(tf.keras.layers.GRU(
      npf2,
      activation = 'tanh',
      dropout=dropout_ss,
      return_sequences=True,
      name='gru2',)
      )
  
  # Third GRU Layer
  npf3 = hp.Int(
      name='neuron_per_feature3',
      min_value=2,
      max_value=200,
      step=1,
      sampling=None,
      default=None,
      parent_name=None,
      parent_values=None,
  )
  
  model.add(tf.keras.layers.GRU(
      npf3,
      activation = 'tanh',
      dropout=dropout_ss,
      return_sequences=False,
      name='gru3',)
      )
  
  # Final Dense Layer Output
  model.add(tf.keras.layers.Dense(1,
      activation='sigmoid',
      name="dense4_output",)
      )
    
  # Configure Adam optimizer
  if optimizer == "adam":
      optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
  elif optimizer == "SGD":
      optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
  else:
      raise("Not supported optimizer")
      
  model.compile(optimizer=optimizer,
                loss=tf.keras.losses.MeanSquaredError(),
                metrics=['mean_squared_error'])
  
  return model

# model_builder(kt.HyperParameters())

# Instantiate the tuner and perform hypertuning

In [22]:
def build_tuner(model, hpo_method, objective, dir_name):
  if hpo_method == "RandomSearch":
      tuner = kt.RandomSearch(model, objective=objective, max_trials=10, executions_per_trial=5,
                              project_name=hpo_method, directory=dir_name)
  elif hpo_method == "Hyperband":
      tuner = kt.Hyperband(model, objective=objective, max_epochs=10, executions_per_trial=5,
                          project_name=hpo_method)
  elif hpo_method == "BayesianOptimization":
      tuner = kt.BayesianOptimization(model, objective=objective, max_trials=10, executions_per_trial=5,
                                      project_name=hpo_method)
  return tuner
  
obj = kt.Objective('mean_squared_error', direction='min')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='mean_squared_error', patience=5)
dir_name = "v1"
randomsearch_tuner = build_tuner(build_model, "Hyperband", obj, dir_name)
randomsearch_tuner.search(dataset, epochs=5, callbacks=[stop_early])

# # Get the optimal hyperparameters
best_hps=randomsearch_tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = build_model(best_hps)
best_model.fit(dataset, epochs=10)

Trial 7 Complete [00h 00m 55s]
mean_squared_error: 0.004667202290147543

Best mean_squared_error So Far: 0.004177630040794611
Total elapsed time: 00h 06m 45s

Search: Running Trial #8

Value             |Best Value So Far |Hyperparameter
tanh              |relu              |activation
adam              |adam              |optimizer
0.00010938        |0.0086419         |learning_rate
0.1               |0.5               |dropouts_
22                |18                |neuron_per_feature1
64                |172               |neuron_per_feature2
168               |180               |neuron_per_feature3
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
128/128 [==============================] - 8s 35ms/step - loss: 0.0047 - mean_squared_error: 0.0047
Epoch 2/2
128/128 [==============================] - 3s 20ms/step - l

KeyboardInterrupt: ignored

In [ ]:
print(best_hps.values)

In [ ]:
best_model.save('./keras_prediction_model_v1')

In [ ]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = best_model.evaluate(dataset, batch_size=25)
print("test loss, test acc:", results)